In [11]:
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.select import Select
from nowatermark import WatermarkRemover
import time
from selenium.common.exceptions import TimeoutException
import os
import random
import pymysql
import re
import redis

In [12]:
def write_file(data, filename):
    with open(filename, 'wb') as f:
        f.write(data)
        f.close()

In [13]:
def Handle_image(image_no_path,image_ok_path):
    remover = WatermarkRemover()
    remover.load_watermark_template('C:/Users/Administrator/Desktop/template.png')
    remover.remove_watermark(image_no_path, image_ok_path)

In [14]:
def Delete_watermark(select_id,image_urls):
        con = pymysql.connect(host='192.168.0.252', user='web_user', passwd='first2018pl,', db='FBDdata2', charset='utf8',
                              port=3306)
        cue = con.cursor()
        path = 'C:/Users/Administrator/Desktop/fql_image/'
        if os.path.exists(path) == False:
            os.mkdir(path)
        shop_path = path + select_id +'/'
        no_path = shop_path + 'is_no/'
        ok_path = shop_path + 'is_ok/'
        if os.path.exists(shop_path) == False:
            os.mkdir(shop_path)
            os.mkdir(no_path)
            os.mkdir(ok_path)
        for image_id in image_urls:
            image_no_path = no_path + image_id + ".jpg"
            image_ok_path = ok_path + image_id + ".jpg"
            print(image_id)
            cue.execute("select image from fqlzu_image where image_id= '%s'" % image_id)
            image = cue.fetchone()[0]
            write_file(image, image_no_path)
            Handle_image(image_no_path, image_ok_path)
        print('图片去水印成功')
        return ok_path

In [15]:
def Read_data():
    
    browser = webdriver.Chrome(r'C:\Users\Administrator\Downloads\chromedriver.exe')
    browser.implicitly_wait(30) # 最长等待30秒
    con = pymysql.connect(
    host='192.168.0.252',
    user='web_user',
    passwd='first2018pl,',
    port=3306,
    db='FBDdata2',
    charset='utf8',
    cursorclass=pymysql.cursors.DictCursor
    )
    
    r = redis.Redis(host='localhost', port=6379)
    redis_data_dict_1 = "fqlzu_id"
    id1 = bytes.decode(r.spop(redis_data_dict_1))
    cursor = con.cursor()
    sql = 'select * from anjuke_shop_data_copy1 WHERE id=%s' % id1
    cursor.execute(sql)
    result = cursor.fetchone()
    status = result['store_status']
    acreage = result['acreage']
    floor = result['floor']
    in_floor = re.search('(\d)层',floor).group(1)
    try:
        all_floor = re.search('共(\d)层',floor).group(1)
    except:
        all_floor = '1'
    width = result['width']
    height = result['height']
    depth = result['depth']
    address = result['address'].split(',')
    try:
        if address[0] == '高新':
            region = '高新区'
        else:
            region = address[0]
    except:
        print('数据不全')
    local = address[1]
    position = address[2]
    monthly_rent = result['monthly_rent']
    payment_method = re.findall('\d',result['payment_method'])
    title = result['title']
    content  = result['content']
    agent_name = result['agent_name']
    agent_phone = result['agent_phone']
    lease_term = result['lease_term']
    image_urls = result['image_urls'].split(',')
    is_facestreet =result['is_facestreet']
    image_ok_path = Delete_watermark(id1,image_urls)
    print(image_ok_path,browser,is_facestreet,acreage,image_urls,status,in_floor,all_floor,width,height,depth,region,local,position,monthly_rent,lease_term,payment_method,title,content,agent_name,agent_phone)
    return image_ok_path,browser,is_facestreet,acreage,image_urls,status,in_floor,all_floor,width,height,depth,region,local,position,monthly_rent,lease_term,payment_method,title,content,agent_name,agent_phone

In [16]:
def Login(browser,username,password):
    browser.get('http://fangqianli.com/')
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="DengL"]').click()
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="username"]').send_keys(username)
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="psw"]').send_keys(password)
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="loginbtn"]').click()
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="app"]/div[1]/div/div[2]/div[6]/a').click()
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="Middle"]/div[1]/div[1]').click()

In [17]:
def Update_image(browser,images_path):
    for i in os.listdir(images_path):
        image = images_path + i
        browser.find_element_by_xpath('//*[@id="Form1"]/div[13]/div[4]/div/div/ul/li/input').send_keys(image)
    time.sleep(3)
    browser.find_element_by_xpath('//*[@id="Form1"]/div[13]/button').click()

In [18]:
def Input_data(browser,status,is_facestreet,acreage,in_floor,all_floor,width,height,depth,region,local,position,monthly_rent,lease_term,payment_method,title,content,agent_name,agent_phone):
    browser.find_element_by_xpath('//*[@id="Form1"]/div[2]/div/div[2]/input').click()
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="Spxuuz"]').click()
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="%s"]' % random.choice((1,2,6)))
    if status == '经营中':
        browser.find_element_by_xpath('//*[@id="Form1"]/div[4]/input[1]').click()
    else:
        browser.find_element_by_xpath('//*[@id="Form1"]/div[4]/input[2]').click()
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="Form1"]/div[6]/input').send_keys(acreage)
    time.sleep(1)
    Select(browser.find_element_by_xpath('//*[@id="foor"]')).select_by_value('dan')
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="Form1"]/div[7]/div[2]/input').send_keys(in_floor)
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="Form1"]/div[7]/div[4]/input').send_keys(all_floor)
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="Form1"]/div[8]/div[1]/input').send_keys(width)
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="Form1"]/div[8]/div[2]/input').send_keys(height)
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="Form1"]/div[8]/div[3]/input').send_keys(depth)
    time.sleep(1)
    if is_facestreet != '临街':
        browser.find_element_by_xpath('//*[@id="Form1"]/div[9]/input[2]').click()
    for i in browser.find_elements_by_name('check'):
        time.sleep(0.5)
        i.click()
    time.sleep(1)
    Select(browser.find_element_by_xpath('//*[@id="selPre"]')).select_by_value(region)
    time.sleep(1)
    Select(browser.find_element_by_xpath('//*[@id="selChi"]')).select_by_value(local)
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="Form1"]/div[11]/input[1]').send_keys(position)
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="Form1"]/div[11]/input[2]').click()
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="Form1"]/div[12]/div[1]/div[1]/input').send_keys(monthly_rent)
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="Form1"]/div[12]/div[1]/div[2]/div[2]/input').send_keys(payment_method[0])
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="Form1"]/div[12]/div[1]/div[2]/div[3]/input').send_keys(payment_method[1])
    if lease_term != '':
        lease = re.findall('\d*',lease_term)[0]
        browser.find_element_by_xpath('//*[@id="Form1"]/div[12]/div[2]/div[1]/input').send_keys(lease)
    else:
        lease = '1'
        print(lease)
        browser.find_element_by_xpath('//*[@id="Form1"]/div[12]/div[2]/div[1]/input').send_keys(lease)      
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="Form1"]/div[12]/div[2]/div[2]/input').send_keys('0')
    browser.find_element_by_xpath('//*[@id="Form1"]/div[13]/div[1]/input').send_keys(title)
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="Form1"]/div[13]/div[2]/div[2]/textarea').send_keys(content)
    time.sleep(1)
    browser.find_element_by_xpath('//*[@id="Form1"]/div[14]/div[1]/input').send_keys(agent_name)
    time.sleep(1)  
    browser.find_element_by_xpath('//*[@id="Form1"]/div[14]/div[2]/input').send_keys(agent_phone)
    

In [19]:
def Run(username,password):
    try:
        image_ok_path,browser,is_facestreet,acreage,image_urls,status,in_floor,all_floor,width,height,depth,region,local,position,monthly_rent,lease_term,payment_method,title,content,agent_name,agent_phone=Read_data()
        time.sleep(3)
        Login(browser,username,password)
        time.sleep(3)
        Input_data(browser,status,is_facestreet,acreage,in_floor,all_floor,width,height,depth,region,local,position,monthly_rent,lease_term,payment_method,title,content,agent_name,agent_phone)
        time.sleep(3)
        Update_image(browser,image_ok_path)
        time.sleep(2)
        browser.find_element_by_xpath('//*[@id="Fabz"]').click()
        time.sleep(3)
    finally:
        browser.quit()

In [21]:
users = [['shopuser','first2018'],['bigger','lp82lf'],['tan2577811','a2577811'],['15756765775','123']]
for user in users:
    Run(user[0],user[1])

34140f13a3e0116ab7c146b626d55ee0
135b8240d71d85708c9233b28868a692
a1372f5c766b538a99a21a5062031c8a
e315b2ad7284be4fbcbcfac7e86a94ea
13c942d2423042fb988ef9a436d76bc9
77f0cf857d8401e007bc7faafd4f328c
图片去水印成功
C:/Users/Administrator/Desktop/fql_image/71107320/is_ok/ <selenium.webdriver.chrome.webdriver.WebDriver (session="1d60e27a906112b54287e11d540829d7")> 临街 91 ['34140f13a3e0116ab7c146b626d55ee0', '135b8240d71d85708c9233b28868a692', 'a1372f5c766b538a99a21a5062031c8a', 'e315b2ad7284be4fbcbcfac7e86a94ea', '13c942d2423042fb988ef9a436d76bc9', '77f0cf857d8401e007bc7faafd4f328c'] 空铺出租/转让 1 1 9 5 10 高新区 孵化园 高新区金融城地铁一号线金融城站 20000 36个月 ['3', '1'] 金融城 中航国际 写字楼底商餐饮旺铺 有天然气 【房源介绍】：1.出租面积：91㎡。———————————2.免租时间：面议———————————3.房屋布局状态：1楼铺面，现在正在经营阁楼已经做好———————————————————————————————————4.商铺优势：城南消费人群集中生活片区，消费实力强劲、金融城地铁1号线716米2孵化园地铁1号线269米周边业态商家名称距离人均消费杀牛场原味老火锅(高新店)18米68.0元包谋天下(奥克斯店)19米27.0元大蓉和酒楼(拉德方斯店)197米78.0元川渝壹号54米49.0元好搭档城门口老火锅(中航店)152米64.0元FOOOO175米90.0元德香苑北京烤鸭(成都店)73米69.0元柴门头啖汤(奥克斯店)93米107.5元恬记Mix耗儿